# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.rag_answerer import IntergraxRagAnswerer, IntergraxAnswererConfig
from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from intergrax.rag.vectorstore_manager import VSConfig, IntergraxVectorstoreManager
from intergrax.rag.re_ranker import ReRankerConfig, IntergraxReRanker
from intergrax.chat_agent import ChatAgentConfig, RagComponent, IntergraxChatAgent
from pydantic import BaseModel, Field



# -----------------------------------------------------------
# 1) LLM + conversational memory
# -----------------------------------------------------------
llm = LLMAdapterRegistry.create(
    name="ollama",
    chat=ChatOllama(model="llama3.1:latest"),
)
memory = IntergraxConversationalMemory()


# -----------------------------------------------------------
# 2) Example Tool: Weather
# -----------------------------------------------------------
class WeatherArgs(BaseModel):
    """Arguments required to call the weather tool."""
    city: str = Field(..., description="City name. Example: 'Warsaw'.")


class WeatherTool(ToolBase):
    """
    Demo weather tool that simulates a weather API response.

    Used by the agent when the user requests weather information.
    """
    name = "get_weather"
    description = (
        "Returns the current weather for a given city. Use only for queries "
        "related to weather, temperature or atmospheric conditions."
    )
    schema_model = WeatherArgs

    def run(self, **kwargs):
        city = kwargs["city"]
        # Static demo response.
        return {"city": city, "tempC": 12.3, "summary": "Partly cloudy"}


# Register available tools
tools = ToolRegistry()
tools.register(WeatherTool())


# -----------------------------------------------------------
# 3) Vector store & RAG configuration
# -----------------------------------------------------------
cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)

store = IntergraxVectorstoreManager(config=cfg, verbose=False)

embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536,
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256,
    ),
    verbose=True,
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=False)


# -----------------------------------------------------------
# 4) RAG Answerer configuration
# -----------------------------------------------------------
cfg = IntergraxAnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)

cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = (
    "Use the following retrieved context to answer the user question: {context}"
)

answerer_docs = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,
    config=cfg,
    memory=None,
    verbose=False,
)

# RAG routing logic description
rag_docs = RagComponent(
    name="intergrax_docs",
    answerer=answerer_docs,
    description=(
        "This component responds to questions related to Mooff regulations, "
        "privacy policies, internal documentation, and compliance rules. "
        "Use this component for legally-bound questions or factual document-based queries."
    ),
    priority=10,
)


# -----------------------------------------------------------
# 5) High-level hybrid agent (RAG + tools + LLM chat)
# -----------------------------------------------------------
agent = IntergraxChatAgent(
    llm=llm,
    memory=memory,
    tools=tools,
    rag_components=[rag_docs],
    config=ChatAgentConfig(verbose=True),
)


# -----------------------------------------------------------
# 6) Test Questions
# -----------------------------------------------------------
for question in [
    "What is the weather in Warsaw?",
    "Explain in detail what Mooff virtual fairs are.",
    "Calculate 22 * 3 and provide just the result.",
]:
    print(question)
    
    response = agent.run(question=question)
    
    print("Routing Decision: ", response["route"])
    print("RAG Component: ", response["rag_component"])
    print("Tool Triggered: ", response["tool_traces"])
    print("Final Answer: ", response["answer"])
    print("=============================================")
